In [2]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import pickle

In [3]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the images to the range 0-1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Reshape dataset to have a single channel
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))

# Convert class vectors to binary class matrices (one-hot encoding)
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

print('MNIST dataset loaded and preprocessed.')
print('Training set size:', x_train.shape, y_train.shape)
print('Test set size:', x_test.shape, y_test.shape)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
MNIST dataset loaded and preprocessed.
Training set size: (60000, 28, 28, 1) (60000, 10)
Test set size: (10000, 28, 28, 1) (10000, 10)


In [4]:
from sklearn.model_selection import train_test_split

# Split the original training data to create a new training set and a validation set
x_train_split, x_val_split, y_train_split, y_val_split = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42)

In [5]:
#  Print the shapes of the splits to verify
print("Shape of new training images:", x_train_split.shape)
print("Shape of new training labels:", y_train_split.shape)
print("Shape of validation images:", x_val_split.shape)
print("Shape of validation labels:", y_val_split.shape)


# Check the shapes of testing data after preprocessing
print("Shape of test images:", x_test.shape)
print("Shape of test labels:", y_test.shape)

Shape of new training images: (48000, 28, 28, 1)
Shape of new training labels: (48000, 10)
Shape of validation images: (12000, 28, 28, 1)
Shape of validation labels: (12000, 10)
Shape of test images: (10000, 28, 28, 1)
Shape of test labels: (10000, 10)


In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, \
Activation, Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model

def resnet_block(input_data, filters, conv_size, activation_func):
    x = Conv2D(filters, conv_size, padding='same')(input_data)
    x = BatchNormalization()(x)
    x = Activation(activation_func)(x)

    x = Conv2D(filters, conv_size, padding='same')(x)
    x = BatchNormalization()(x)

    # Adding the input data to the output of the block (Skip Connection)
    x = Add()([x, input_data])

    x = Activation(activation_func)(x)
    return x

def build_resnet20(input_shape, num_classes, activation_func):
    inputs = Input(shape=input_shape)

    # Initial Conv Layer
    x = Conv2D(16, (3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation(activation_func)(x)

    # ResNet Blocks
    for _ in range(3):
        x = resnet_block(x, 16, (3, 3), activation_func)

    # Transition Layer
    x = Conv2D(32, (3, 3), padding='same', strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Activation(activation_func)(x)

    for _ in range(3):
        x = resnet_block(x, 32, (3, 3), activation_func)

    # Transition Layer
    x = Conv2D(64, (3, 3), padding='same', strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Activation(activation_func)(x)

    for _ in range(3):
        x = resnet_block(x, 64, (3, 3), activation_func)

    # Average Pooling
    x = AveragePooling2D(pool_size=(7, 7))(x)
    x = Flatten()(x)

    # Output Layer
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Custom activation function
def custom_activation(x):
    # Define your custom activation logic here
    return tf.nn.relu(x)  # Example: using ReLU as a placeholder

# Building the model with the custom activation function
input_shape = (32, 32, 3)  # Change based on your dataset
num_classes = 10  # Change based on your dataset

model = build_resnet20(input_shape, num_classes, custom_activation)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 32, 32,    │        448 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 32, 32,    │         64 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 32, 32,    │      2,320 │ activation[0][0]  │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │         64 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 32, 32,    │      2,320 │ activation_1[0][… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │         64 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 32, 32,    │          0 │ batch_normalizat… │
│                     │ 16)               │            │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 32, 32,    │          0 │ add[0][0]         │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 32, 32,    │      2,320 │ activation_2[0][… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │         64 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 32, 32,    │      2,320 │ activation_3[0][… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │         64 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 32, 32,    │          0 │ batch_normalizat

 Total params: 318,346 (1.21 MB)

 Trainable params: 316,778 (1.21 MB)

 Non-trainable params: 1,568 (6.12 KB)

In [10]:
def train_model(activation_func, x_train, y_train, x_val, y_val, batch_size, learning_rate, name):
    # Build the model
    model = build_resnet20(input_shape=x_train.shape[1:], num_classes=y_train.shape[1], activation_func=activation_func)

    # Compile the model with specified learning rate
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy',
                  metrics=[
                  'accuracy',
                  tf.keras.metrics.Precision(name='precision'),
                  tf.keras.metrics.Recall(name='recall'),
                  tf.keras.metrics.AUC(name='auc')
              ])
    
        # Define the checkpoint callback
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
        f'{name}.keras', # Path where to save the model
        save_best_only=True, # Only save a model if `val_loss` has improved
        save_weights_only = False,
        monitor='val_loss', # Monitor the validation loss
        mode='min', # The lower the validation loss, the better the model
        verbose=1 # Log a message when a better model is found
    )


    # Train the model with specified batch size
    history = model.fit(x_train, y_train, epochs=60, batch_size=batch_size,
                        validation_data=(x_val, y_val), verbose=1, callbacks = [checkpoint_cb])

    return history, model



# Parameters
batch_size = 32
learning_rate = 0.005

# Activation functions to try
activation_functions = [tf.nn.relu, tf.nn.sigmoid, tf.nn.tanh]
names = ["relu", "sigmoid", "tanh"]
histories = {}

# Train and evaluate the model with each activation function
print(f"Training with Relu activation function")
history_relu, model_relu = train_model(tf.nn.relu, x_train_split, y_train_split,
                          x_val_split, y_val_split, batch_size, learning_rate, "relu")
histories["relu"] = history_relu

print(f"\n\n Training with Sigmoid activation function")
history_sigmoid, model_sig = train_model( tf.nn.sigmoid, x_train_split, y_train_split,
                          x_val_split, y_val_split, batch_size, learning_rate, "sigmoid")
histories["sigmoid"] = history_sigmoid


print(f"\n\n Training with Tanh activation function")
history_tanh, model_tanh = train_model(tf.nn.tanh, x_train_split, y_train_split,
                          x_val_split, y_val_split, batch_size, learning_rate, "tanh")
histories["tanh"] = history_tanh

Training with Relu activation function
Epoch 1/60
1495/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9043 - auc: 0.9879 - loss: 0.3089 - precision: 0.9421 - recall: 0.8771
Epoch 1: val_loss improved from inf to 0.17684, saving model to relu.keras
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 38s 12ms/step - accuracy: 0.9045 - auc: 0.9879 - loss: 0.3082 - precision: 0.9422 - recall: 0.8774 - val_accuracy: 0.9542 - val_auc: 0.9949 - val_loss: 0.1768 - val_precision: 0.9576 - val_recall: 0.9517
Epoch 2/60
1496/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9799 - auc: 0.9988 - loss: 0.0678 - precision: 0.9821 - recall: 0.9776
Epoch 2: val_loss improved from 0.17684 to 0.07466, saving model to relu.keras
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9799 - auc: 0.9988 - loss: 0.0678 - precision: 0.9821 - recall: 0.9776 - val_accuracy: 0.9786 - val_auc: 0.9981 - val_loss: 0.0747 - val_precision: 0.9817 - val_recall: 0.9766
Epoch 3/60
1498/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - acc

In [11]:
def print_info(history, model_name):
    history = history.history
    
    print("*" * 50)
    print(f"\n{model_name} Results:")
    print("*" * 50)
    print("\n")

    # Assuming history['loss'], history['val_loss'], etc., exist
    training_loss = history['loss']
    validation_loss = history['val_loss']
    training_accuracy = history['accuracy']
    validation_accuracy = history['val_accuracy']
    training_auc = history['auc']
    validation_auc = history['val_auc']
    training_precision = history['precision']
    validation_precision = history['val_precision']
    training_recall = history['recall']
    validation_recall = history['val_recall']
    


    # Calculate F1 scores based on available precision and recall in history
    training_f1 = calculate_f1_scores(history['precision'], history['recall'])
    validation_f1 = calculate_f1_scores(history['val_precision'], history['val_recall'])


    top_3_dict[model_name] = {
        "training_loss": sorted(training_loss)[:3],
        "validation_loss": sorted(validation_loss)[:3],
        "training_accuracy": sorted(training_accuracy, reverse=True)[:3],
        "validation_accuracy": sorted(validation_accuracy, reverse=True)[:3],
        "training_auc": sorted(training_auc, reverse=True)[:3],
        "validation_auc": sorted(validation_auc, reverse=True)[:3], 
        "training_precision": sorted(training_precision, reverse=True)[:3],
        "validation_precision": sorted(validation_precision, reverse=True)[:3],
        "training_recall": sorted(training_recall, reverse=True)[:3],
        "validation_recall": sorted(validation_recall, reverse=True)[:3], 
        "training_f1": sorted(training_f1, reverse=True)[:3],
        "validation_f1": sorted(validation_f1, reverse=True)[:3]

    }

    # Print Top 3 Lowest Losses
    print("Top 3 Lowest Training Losses:", sorted(training_loss)[:3])
    print("Top 3 Lowest Validation Losses:", sorted(validation_loss)[:3])

    # Print Top 3 Highest Accuracies
    print("Top 3 Highest Training Accuracies:", sorted(training_accuracy, reverse=True)[:3])
    print("Top 3 Highest Validation Accuracies:", sorted(validation_accuracy, reverse=True)[:3])

    # Print Top 3 AUCs
    print("Top 3 Training AUCs:", sorted(training_auc, reverse=True)[:3])
    print("Top 3 Validation AUCs:", sorted(validation_auc, reverse=True)[:3])

    # Print Top 3 F1 Scores
    print("Top 3 Training F1 Scores:", sorted(training_f1, reverse=True)[:3])
    print("Top 3 Validation F1 Scores:", sorted(validation_f1, reverse=True)[:3])

    # Print Top 3 Precision
    print("Top 3 Training Precision:", sorted(training_precision, reverse=True)[:3])
    print("Top 3 Validation Precision:", sorted(validation_precision, reverse=True)[:3])

    # Print Top 3 Recall
    print("Top 3 Training Recall:", sorted(training_recall, reverse=True)[:3])
    print("Top 3 Validation Recall:", sorted(validation_recall, reverse=True)[:3])

In [17]:
print_info(history_relu, "RELU")
print_info(history_sigmoid, "SIGMOID")
print_info(history_tanh, "TANH")

**************************************************

RELU Results:
**************************************************


Top 3 Lowest Training Losses: [0.00034027054789476097, 0.00044127850560471416, 0.00095847446937114]
Top 3 Lowest Validation Losses: [0.02439233846962452, 0.02517787739634514, 0.026302499696612358]
Top 3 Highest Training Accuracies: [0.999916672706604, 0.9998958110809326, 0.9997291564941406]
Top 3 Highest Validation Accuracies: [0.9947500228881836, 0.9944999814033508, 0.9944166541099548]
Top 3 Training AUCs: [1.0, 1.0, 0.9999881982803345]
Top 3 Validation AUCs: [0.9994715452194214, 0.9993517398834229, 0.9993330836296082]
Top 3 Training F1 Scores: [0.999916672706604, 0.9998958110809326, 0.9997395871981462]
Top 3 Validation F1 Scores: [0.994791446391268, 0.9944990894975356, 0.9944157324094369]
Top 3 Training Precision: [0.999916672706604, 0.9998958110809326, 0.999750018119812]
Top 3 Validation Precision: [0.9948328733444214, 0.9946649074554443, 0.9945815205574036]
Top 3 T

In [7]:
import tensorflow as tf


class SmoothTransitionReLU(tf.keras.layers.Layer):
    def __init__(self, initial_slope, final_slope, steepness=10, **kwargs):
        super(SmoothTransitionReLU, self).__init__(**kwargs)
        self.initial_slope = initial_slope
        self.final_slope = final_slope
        self.steepness = steepness
        # Internal counter to track the relative progress of training
        self.progress = tf.Variable(0.0, trainable=False, dtype=tf.float32)

    def call(self, inputs, training=None):
        if training:
            # Increment the progress during training (you might need to adjust how this increments based on your training regime)
            self.progress.assign_add(0.01)  # Increment by a small value on each call

        # Calculate the current slope based on the sigmoid function
        x = self.progress
        current_slope = self.initial_slope + (self.final_slope - self.initial_slope) / (1 + tf.exp(-self.steepness * (x - 0.5)))

        # Apply the dynamic slope to the positive part of the inputs
        positive_part = tf.maximum(0.0, inputs) * current_slope
        # For negative inputs, just pass them through or adjust as needed
        negative_part = tf.minimum(0.0, inputs)

        return positive_part + negative_part

    def get_config(self):
        config = super(SmoothTransitionReLU, self).get_config()
        config.update({
            "initial_slope": self.initial_slope,
            "final_slope": self.final_slope,
            "steepness": self.steepness
        })
        return config

In [8]:
def train_model_with_custom_activation(x_train, y_train, x_val, y_val, batch_size, learning_rate,
                                       initial_slope, target_slope, total_epochs):
    # Initialize the custom activation function with provided parameters
    custom_activation = SmoothTransitionReLU(initial_slope=initial_slope, final_slope=target_slope)

    # Build the model using the custom activation function
    model = build_resnet20(input_shape=x_train.shape[1:], num_classes=y_train.shape[1],
                        activation_func=custom_activation)

    # Define the checkpoint callback
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
        'dynamic_relu_model.keras',
        save_best_only=True,
        monitor='val_loss',
        mode='min',
        verbose=1
    )

    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', 
                  metrics=['accuracy', 'precision', 'recall', 'auc'])

    # Train the model
    history = model.fit(x_train, y_train, epochs=total_epochs, batch_size=batch_size,
                        validation_data=(x_val, y_val), verbose=1,
                        callbacks=[checkpoint_cb])

    return history, model


# Example parameters
batch_size = 32
learning_rate = 0.005
initial_slope = 1.732
target_slope = 0.557
rate = 0.01
total_epochs = 60

# Train the model
history_custom, model_custom = train_model_with_custom_activation(
    x_train_split, y_train_split, x_val_split, y_val_split, batch_size, learning_rate,
    initial_slope, target_slope, total_epochs
)

histories = {}
histories["custom"] = history_custom

Epoch 1/60
   8/1500 ━━━━━━━━━━━━━━━━━━━━ 24s 16ms/step - accuracy: 0.1405 - auc: 0.5384 - loss: 4.2130 - precision: 0.1382 - recall: 0.0481           

I0000 00:00:1710586766.627562     108 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1498/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8515 - auc: 0.9762 - loss: 0.4917 - precision: 0.9063 - recall: 0.8037
Epoch 1: val_loss improved from inf to 0.22246, saving model to dynamic_relu_model.keras
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 44s 12ms/step - accuracy: 0.8517 - auc: 0.9762 - loss: 0.4912 - precision: 0.9064 - recall: 0.8039 - val_accuracy: 0.9388 - val_auc: 0.9930 - val_loss: 0.2225 - val_precision: 0.9428 - val_recall: 0.9357
Epoch 2/60
1496/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9698 - auc: 0.9985 - loss: 0.0984 - precision: 0.9739 - recall: 0.9658
Epoch 2: val_loss improved from 0.22246 to 0.12127, saving model to dynamic_relu_model.keras
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9698 - auc: 0.9985 - loss: 0.0984 - precision: 0.9739 - recall: 0.9659 - val_accuracy: 0.9667 - val_auc: 0.9967 - val_loss: 0.1213 - val_precision: 0.9686 - val_recall: 0.9651
Epoch 3/60
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9777 - auc: 0

In [9]:
# Function to calculate F1 scores from precision and recall
def calculate_f1_scores(precision, recall):
    return 2 * (np.array(precision) * np.array(recall)) / (np.array(precision) + np.array(recall))

top_3_dict = {}
test = {}
# histories = dict(histories["custom"].history)
test["custom"] = histories["custom"].history


for model_name, history in test.items():
    print(f"\n{model_name} Results:")

    # Assuming history['loss'], history['val_loss'], etc., exist
    training_loss = history['loss']
    validation_loss = history['val_loss']
    training_accuracy = history['accuracy']
    validation_accuracy = history['val_accuracy']
    training_auc = history['auc']
    validation_auc = history['val_auc']
    training_precision = history['precision']
    validation_precision = history['val_precision']
    training_recall = history['recall']
    validation_recall = history['val_recall']
    


    # Calculate F1 scores based on available precision and recall in history
    training_f1 = calculate_f1_scores(history['precision'], history['recall'])
    validation_f1 = calculate_f1_scores(history['val_precision'], history['val_recall'])


    top_3_dict[model_name] = {
        "training_loss": sorted(training_loss)[:3],
        "validation_loss": sorted(validation_loss)[:3],
        "training_accuracy": sorted(training_accuracy, reverse=True)[:3],
        "validation_accuracy": sorted(validation_accuracy, reverse=True)[:3],
        "training_auc": sorted(training_auc, reverse=True)[:3],
        "validation_auc": sorted(validation_auc, reverse=True)[:3], 
        "training_precision": sorted(training_precision, reverse=True)[:3],
        "validation_precision": sorted(validation_precision, reverse=True)[:3],
        "training_recall": sorted(training_recall, reverse=True)[:3],
        "validation_recall": sorted(validation_recall, reverse=True)[:3], 
        "training_f1": sorted(training_f1, reverse=True)[:3],
        "validation_f1": sorted(validation_f1, reverse=True)[:3]

    }

    # Print Top 3 Lowest Losses
    print("Top 3 Lowest Training Losses:", sorted(training_loss)[:3])
    print("Top 3 Lowest Validation Losses:", sorted(validation_loss)[:3])

    # Print Top 3 Highest Accuracies
    print("Top 3 Highest Training Accuracies:", sorted(training_accuracy, reverse=True)[:3])
    print("Top 3 Highest Validation Accuracies:", sorted(validation_accuracy, reverse=True)[:3])

    # Print Top 3 AUCs
    print("Top 3 Training AUCs:", sorted(training_auc, reverse=True)[:3])
    print("Top 3 Validation AUCs:", sorted(validation_auc, reverse=True)[:3])

    # Print Top 3 F1 Scores
    print("Top 3 Training F1 Scores:", sorted(training_f1, reverse=True)[:3])
    print("Top 3 Validation F1 Scores:", sorted(validation_f1, reverse=True)[:3])

    # Print Top 3 Precision
    print("Top 3 Training Precision:", sorted(training_precision, reverse=True)[:3])
    print("Top 3 Validation Precision:", sorted(validation_precision, reverse=True)[:3])

    # Print Top 3 Recall
    print("Top 3 Training Recall:", sorted(training_recall, reverse=True)[:3])
    print("Top 3 Validation Recall:", sorted(validation_recall, reverse=True)[:3])


custom Results:
Top 3 Lowest Training Losses: [0.003998010419309139, 0.0048003895208239555, 0.004837052430957556]
Top 3 Lowest Validation Losses: [0.029127471148967743, 0.03208880499005318, 0.03524142503738403]
Top 3 Highest Training Accuracies: [0.9987916946411133, 0.9985833168029785, 0.9984583258628845]
Top 3 Highest Validation Accuracies: [0.9925833344459534, 0.9921666383743286, 0.9921666383743286]
Top 3 Training AUCs: [0.9999642968177795, 0.9999637603759766, 0.9999528527259827]
Top 3 Validation AUCs: [0.9994426369667053, 0.9994312524795532, 0.9992927312850952]
Top 3 Training F1 Scores: [0.9987812015127027, 0.9985415896514309, 0.9984791261503784]
Top 3 Validation F1 Scores: [0.992706821967684, 0.9923314129581963, 0.9922906899302029]
Top 3 Training Precision: [0.9988332390785217, 0.9986039996147156, 0.9985207319259644]
Top 3 Validation Precision: [0.9929137229919434, 0.9925796389579773, 0.9924147725105286]
Top 3 Training Recall: [0.9987291693687439, 0.9984791874885559, 0.99843752384

In [13]:
def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))

In [ ]:
history_mish, model_mish = train_model(mish, x_train_split, y_train_split,
                          x_val_split, y_val_split, batch_size, learning_rate, "mish")
histories["mish"] = history_mish

Epoch 1/60
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9119 - auc: 0.9890 - loss: 0.2891 - precision: 0.9450 - recall: 0.8806
Epoch 1: val_loss improved from inf to 0.15094, saving model to mish.keras
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 43s 13ms/step - accuracy: 0.9119 - auc: 0.9890 - loss: 0.2890 - precision: 0.9450 - recall: 0.8806 - val_accuracy: 0.9617 - val_auc: 0.9950 - val_loss: 0.1509 - val_precision: 0.9646 - val_recall: 0.9594
Epoch 2/60
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9833 - auc: 0.9991 - loss: 0.0569 - precision: 0.9853 - recall: 0.9816
Epoch 2: val_loss improved from 0.15094 to 0.10609, saving model to mish.keras
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.9833 - auc: 0.9991 - loss: 0.0569 - precision: 0.9853 - recall: 0.9816 - val_accuracy: 0.9713 - val_auc: 0.9974 - val_loss: 0.1061 - val_precision: 0.9739 - val_recall: 0.9699
Epoch 3/60
1497/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9848 - auc: 0.9993 - loss: 

In [18]:
history_Save = {
    "relu" : histories["relu"].history, 
    "sigmoid" : histories["sigmoid"].history, 
    "tanh" : histories["tanh"].history, 
    "custom" : histories["custom"].history, 
    "mish" : histories["mish"].history}

In [19]:
# Dumping the dictionary into a pickle file
file_path = "history_mitbih.pkl"
with open(file_path, 'wb') as file:
    pickle.dump(history_Save, file)

file_path

'history_mitbih.pkl'